In [98]:
import pandas as pd
from sqlalchemy import create_engine
import json
from pandas.io.json import json_normalize

In [99]:
csv_file = "./Resources/USvideos.csv"
youtube_data_df = pd.read_csv(csv_file, usecols=['video_id', 'title', 'views', 'tags'])

youtube_data_df.head()

,video_id,title,tags,views
0,2kyS6SvSYSE,WE WANT TO TALK ABOUT OUR MARRIAGE,SHANtell martin,748374
1,1ZAPwfrtAFY,The Trump Presidency: Last Week Tonight with J...,"last week tonight trump presidency|""last week ...",2418783
2,5qpjK5DgCt4,"Racist Superman | Rudy Mancuso, King Bach & Le...","racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434
3,puqaWrEC7tY,Nickelback Lyrics: Real or Fake?,"rhett and link|""gmm""|""good mythical morning""|""...",343168
4,d380meD0W0M,I Dare You: GOING BALD!?,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731


In [100]:
json_file = './Resources/trending.json'
with open(json_file, 'r', encoding='utf-8') as j:
     data = json.loads(j.read())

tiktok_data_df = pd.DataFrame(data['collector'], columns=['id', 'text', 'playCount', 'hashtags'])
tiktok_data_df.head()

,id,text,playCount,hashtags
0,6907228749016714497,Confidence went 📈,44800,[]
1,6875468410612993286,Quiet Zone... follow me on insta: joeysofo. Co...,838100,[]
2,6898699405898059010,Iphone bend test🤗 #tiktok #viral #fyp #iphone ...,15300000,"[{'id': '23428', 'name': 'tiktok', 'title': ''..."
3,6902819837345533186,,94900,[]
4,6905635666588192002,小技です👟✨#tiktok教室#tutorial,115300,"[{'id': '1627704798586882', 'name': 'tiktok教室'..."


In [106]:
test_tiktok_data_df = pd.json_normalize(data['collector'], record_path="hashtags", meta=['id', 'text', 'playCount'])
test_tiktok_data_df.head()

ValueError: Conflicting metadata name id, need distinguishing prefix 

In [105]:
FIELDS = ["id", "text", "playCount", "hashtags"]
json_normalize(data['collector'])
test_tiktok_data_df = pd.json_normalize(data['collector'])
list(test_tiktok_data_df.columns)
# test_tiktok_data_df[FIELDS].head()
# test_tiktok_data_df.head()

TypeError: list indices must be integers or slices, not str

In [103]:
# Connect to Database 
rds_connection_string = "trends_project:password@localhost:5432/trending_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [104]:
engine.table_names() 

<ipython-input-104-8f1edc7208d0>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


OperationalError: (psycopg2.OperationalError) FATAL:  password authentication failed for user "trends_project"

(Background on this error at: http://sqlalche.me/e/14/e3q8)

In [37]:
new_youtube_df = youtube_data_df.rename(columns={"video_id": "platform_id", "views": "view_count"})

new_youtube_df.head()

,platform_id,title,tags,view_count
0,2kyS6SvSYSE,WE WANT TO TALK ABOUT OUR MARRIAGE,SHANtell martin,748374
1,1ZAPwfrtAFY,The Trump Presidency: Last Week Tonight with J...,"last week tonight trump presidency|""last week ...",2418783
2,5qpjK5DgCt4,"Racist Superman | Rudy Mancuso, King Bach & Le...","racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434
3,puqaWrEC7tY,Nickelback Lyrics: Real or Fake?,"rhett and link|""gmm""|""good mythical morning""|""...",343168
4,d380meD0W0M,I Dare You: GOING BALD!?,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731


In [69]:
new_youtube_df['tags'].str.split('|')

0                                        [SHANtell martin]
1        [last week tonight trump presidency, "last wee...
2        [racist superman, "rudy", "mancuso", "king", "...
3        [rhett and link, "gmm", "good mythical morning...
4        [ryan, "higa", "higatv", "nigahiga", "i dare y...
                               ...                        
40944    [aarons animals, "aarons", "animals", "cat", "...
40945                                             [[none]]
40946    [I gave safiya nygaard a perfect hair makeover...
40947    [Black Panther, "HISHE", "Marvel", "Infinity W...
40948    [call of duty, "cod", "activision", "Black Ops...
Name: tags, Length: 40949, dtype: object

In [70]:
new_tiktok_data_df = tiktok_data_df.rename(columns={"id": "platform_id", "text": "title", "hashtags": "tags", "playCount": "view_count"})
new_tiktok_data_df.head()

,platform_id,title,view_count,tags
0,6907228749016714497,Confidence went 📈,44800,[]
1,6875468410612993286,Quiet Zone... follow me on insta: joeysofo. Co...,838100,[]
2,6898699405898059010,Iphone bend test🤗 #tiktok #viral #fyp #iphone ...,15300000,"[{'id': '23428', 'name': 'tiktok', 'title': ''..."
3,6902819837345533186,,94900,[]
4,6905635666588192002,小技です👟✨#tiktok教室#tutorial,115300,"[{'id': '1627704798586882', 'name': 'tiktok教室'..."


In [74]:
clean_tiktok_df = pd.json_normalize(new_tiktok_data_df['tags'])
clean_tiktok_df.head()

AttributeError: 'list' object has no attribute 'values'

In [65]:
new_tiktok_data_df = tiktok_data_df.rename(columns={"id": "platform_id", "text": "title", "hashtags": "tags", "playCount": "view_count"})
# tags = new_tiktok_data_df['tags']
new_tiktok_data_df['tags'].str.split('-',expand=True)

for index, tags in new_tiktok_data_df[['platform_id', 'tags']].iterrows():
    for i, val in enumerate(tags):
        for index, j in enumerate(val):    
            print(index, " ", j)
#         print (val[1['name']])
#     for item in tags: 
#         print(item.)
#     print(tags)

# tags.head()

0   6
1   9
2   0
3   7
4   2
5   2
6   8
7   7
8   4
9   9
10   0
11   1
12   6
13   7
14   1
15   4
16   4
17   9
18   7
0   6
1   8
2   7
3   5
4   4
5   6
6   8
7   4
8   1
9   0
10   6
11   1
12   2
13   9
14   9
15   3
16   2
17   8
18   6
0   6
1   8
2   9
3   8
4   6
5   9
6   9
7   4
8   0
9   5
10   8
11   9
12   8
13   0
14   5
15   9
16   0
17   1
18   0
0   {'id': '23428', 'name': 'tiktok', 'title': '', 'cover': ''}
1   {'id': '20884', 'name': 'viral', 'title': '', 'cover': ''}
2   {'id': '229207', 'name': 'fyp', 'title': '', 'cover': ''}
3   {'id': '14193', 'name': 'iphone', 'title': '', 'cover': ''}
4   {'id': '15134', 'name': 'test', 'title': '', 'cover': ''}
5   {'id': '16962', 'name': 'bend', 'title': '', 'cover': ''}
0   6
1   9
2   0
3   2
4   8
5   1
6   9
7   8
8   3
9   7
10   3
11   4
12   5
13   5
14   3
15   3
16   1
17   8
18   6
0   6
1   9
2   0
3   5
4   6
5   3
6   5
7   6
8   6
9   6
10   5
11   8
12   8
13   1
14   9
15   2
16   0
17   0
18   2
0   {'id

11   3
12   0
13   5
14   9
15   8
16   9
17   1
18   7
0   {'id': '26239', 'name': 'checkitout', 'title': '', 'cover': ''}
0   6
1   9
2   0
3   7
4   2
5   3
6   3
7   5
8   3
9   4
10   8
11   9
12   3
13   6
14   4
15   0
16   9
17   6
18   5
0   {'id': '38180458', 'name': 'horadearrasar', 'title': 'Use o novo Modelo de Vídeo (MV) do TikTok e arrase na trend com suas melhores fotos! 😎', 'cover': 'https://p77-sg.tiktokcdn.com/obj/tiktok-obj/4be9ef52956daf3f357575a507ffbd2a'}
1   {'id': '153828', 'name': 'fy', 'title': '', 'cover': ''}
2   {'id': '42164', 'name': 'foryou', 'title': '', 'cover': ''}
3   {'id': '23428', 'name': 'tiktok', 'title': '', 'cover': ''}
0   6
1   8
2   7
3   6
4   7
5   6
6   0
7   4
8   2
9   7
10   2
11   2
12   9
13   9
14   5
15   7
16   3
17   7
18   7
0   {'id': '38180458', 'name': 'horadearrasar', 'title': 'Use o novo Modelo de Vídeo (MV) do TikTok e arrase na trend com suas melhores fotos! 😎', 'cover': 'https://p77-sg.tiktokcdn.com/obj/tiktok-obj/4be9

4   5
5   0
6   2
7   5
8   1
9   3
10   4
11   6
12   9
13   4
14   1
15   0
16   5
17   6
18   5
0   {'id': '400687', 'name': 'animeedit', 'title': '', 'cover': ''}
1   {'id': '5917', 'name': 'anime', 'title': '', 'cover': ''}
2   {'id': '188522', 'name': 'weeb', 'title': '', 'cover': ''}
3   {'id': '4231', 'name': 'love', 'title': '', 'cover': ''}
4   {'id': '5163', 'name': 'family', 'title': '', 'cover': ''}
5   {'id': '5433', 'name': 'friends', 'title': '#FriendshipDay Contest: Create a musical with some #friends and share it on Instagram and here. The best ones get featured.', 'cover': ''}
6   {'id': '783626', 'name': 'recommendations', 'title': '', 'cover': ''}
7   {'id': '5243', 'name': 'workout', 'title': '', 'cover': ''}
8   {'id': '7882', 'name': 'gym', 'title': '', 'cover': ''}
9   {'id': '66006', 'name': 'weight', 'title': '', 'cover': ''}
10   {'id': '6888', 'name': 'relatable', 'title': '', 'cover': ''}
11   {'id': '17641', 'name': 'facts', 'title': '', 'cover': ''}
12  

0   {'id': '38180458', 'name': 'horadearrasar', 'title': 'Use o novo Modelo de Vídeo (MV) do TikTok e arrase na trend com suas melhores fotos! 😎', 'cover': 'https://p77-sg.tiktokcdn.com/obj/tiktok-obj/4be9ef52956daf3f357575a507ffbd2a'}
1   {'id': '1639598605846533', 'name': 'ambev', 'title': '', 'cover': ''}
2   {'id': '23428', 'name': 'tiktok', 'title': '', 'cover': ''}
3   {'id': '3069639', 'name': 'brahma', 'title': '', 'cover': ''}
4   {'id': '38180458', 'name': 'horadearrasar', 'title': 'Use o novo Modelo de Vídeo (MV) do TikTok e arrase na trend com suas melhores fotos! 😎', 'cover': 'https://p77-sg.tiktokcdn.com/obj/tiktok-obj/4be9ef52956daf3f357575a507ffbd2a'}
0   6
1   9
2   0
3   5
4   5
5   3
6   0
7   0
8   0
9   5
10   7
11   1
12   1
13   7
14   6
15   8
16   8
17   3
18   3
0   6
1   9
2   0
3   4
4   3
5   9
6   1
7   9
8   3
9   4
10   7
11   3
12   5
13   9
14   4
15   4
16   9
17   6
18   1
0   {'id': '3517681', 'name': 'gemelas', 'title': '', 'cover': ''}
1   {'id': 

17   7
18   7
0   6
1   8
2   9
3   9
4   4
5   4
6   1
7   8
8   9
9   0
10   6
11   7
12   7
13   1
14   1
15   6
16   1
17   6
18   2
0   {'id': '419157', 'name': 'shapeshifter', 'title': '', 'cover': ''}
1   {'id': '1030900', 'name': 'shapeshifting', 'title': '', 'cover': ''}
0   6
1   9
2   0
3   6
4   7
5   4
6   1
7   8
8   9
9   0
10   5
11   0
12   3
13   1
14   8
15   3
16   6
17   2
18   1
0   {'id': '1681950897702917', 'name': 'toolstour', 'title': '', 'cover': ''}
1   {'id': '11417614', 'name': 'drillbit', 'title': '', 'cover': ''}
2   {'id': '2897906', 'name': 'electrician', 'title': '', 'cover': ''}
3   {'id': '85077', 'name': 'tools', 'title': '', 'cover': ''}
4   {'id': '1604894787222533', 'name': 'amazonfinds', 'title': '', 'cover': ''}
5   {'id': '1603031833008133', 'name': 'homedep', 'title': '', 'cover': ''}
0   6
1   8
2   9
3   9
4   7
5   8
6   4
7   9
8   8
9   8
10   0
11   9
12   5
13   4
14   9
15   9
16   5
17   2
18   2
0   6
1   8
2   9
3   3
4   1
5   8


...

1   8
2   9
3   4
4   3
5   1
6   6
7   5
8   9
9   1
10   4
11   5
12   0
13   7
14   0
15   5
16   1
17   5
18   4
0   {'id': '42164', 'name': 'foryou', 'title': '', 'cover': ''}
1   {'id': '573', 'name': 'taylorswift', 'title': '', 'cover': ''}
2   {'id': '3506245', 'name': 'holidaycountdown', 'title': 'How are you counting down the days until the holidays?', 'cover': ''}
3   {'id': '47298', 'name': 'transition', 'title': '', 'cover': ''}
4   {'id': '38103', 'name': 'swiftie', 'title': '', 'cover': ''}
5   {'id': '229207', 'name': 'fyp', 'title': '', 'cover': ''}
0   6
1   9
2   0
3   3
4   9
5   7
6   4
7   8
8   8
9   4
10   1
11   5
12   4
13   6
14   0
15   8
16   8
17   9
18   7
0   {'id': '42164', 'name': 'foryou', 'title': '', 'cover': ''}
1   {'id': '153828', 'name': 'fy', 'title': '', 'cover': ''}
0   6
1   8
2   7
3   7
4   6
5   0
6   6
7   7
8   5
9   0
10   9
11   0
12   0
13   5
14   3
15   8
16   6
17   2
18   6
0   {'id': '153828', 'name': 'fy', 'title': '', 'cover':

5   3
6   6
7   4
8   9
9   1
10   5
11   3
12   6
13   9
14   4
15   4
16   3
17   8
18   6
0   {'id': '696879', 'name': 'pijn', 'title': '', 'cover': ''}
1   {'id': '1053376', 'name': 'ziekenhuis', 'title': '', 'cover': ''}
2   {'id': '6830763', 'name': 'voorjou', 'title': '', 'cover': ''}
3   {'id': '5329840', 'name': 'middelbare', 'title': '', 'cover': ''}
4   {'id': '4762782', 'name': 'sporten', 'title': '', 'cover': ''}
5   {'id': '3264520', 'name': 'blessure', 'title': '', 'cover': ''}
6   {'id': '1663974615548934', 'name': 'jouwverhaal', 'title': '', 'cover': ''}
7   {'id': '10842462', 'name': 'verhalen', 'title': '', 'cover': ''}
0   6
1   9
2   0
3   3
4   1
5   8
6   8
7   9
8   1
9   8
10   0
11   7
12   8
13   8
14   5
15   2
16   3
17   5
18   4
0   {'id': '12936964', 'name': 'mallplaza', 'title': '', 'cover': ''}
1   {'id': '29677', 'name': 'baile', 'title': '', 'cover': ''}
2   {'id': '24324', 'name': 'siguemeytesigo', 'title': '', 'cover': ''}
3   {'id': '65243', 'name

9   0
10   5
11   3
12   4
13   3
14   3
15   0
16   6
17   2
18   5
0   {'id': '18121', 'name': 'diy', 'title': 'Do it yourself!', 'cover': 'https://p16-sg.tiktokcdn.com/obj/musically-maliva-obj/1baf2e302b5f32ac74476e1499dff267'}
1   {'id': '456806', 'name': 'tutorials', 'title': '', 'cover': ''}
2   {'id': '6469', 'name': 'follow', 'title': '', 'cover': ''}
3   {'id': '38170190', 'name': 'diyhome', 'title': '', 'cover': ''}
4   {'id': '84217102', 'name': 'diyhomedecor', 'title': '', 'cover': ''}
5   {'id': '8166', 'name': 'tutorial', 'title': '#AprendeEnTikTok cómo hacer lo que sea! Muéstranos un #Tutorial que tenemos que aprender! 🙌', 'cover': 'https://p16-sg.tiktokcdn.com/obj/musically-maliva-obj/eca4f0ac54dc4ed1b2cf76ee6566cdc0'}
6   {'id': '99690', 'name': 'makeover', 'title': '', 'cover': ''}
7   {'id': '8016', 'name': 'gold', 'title': '', 'cover': ''}
8   {'id': '22642', 'name': 'creative', 'title': '', 'cover': ''}
9   {'id': '5222', 'name': 'like', 'title': '', 'cover': ''}
0

15   8
16   0
17   9
18   7
0   6
1   8
2   9
3   6
4   4
5   4
6   9
7   1
8   3
9   7
10   4
11   2
12   0
13   6
14   1
15   9
16   0
17   0
18   9
0   {'id': '55704', 'name': 'dutch', 'title': '', 'cover': ''}
1   {'id': '6830763', 'name': 'voorjou', 'title': '', 'cover': ''}
2   {'id': '229207', 'name': 'fyp', 'title': '', 'cover': ''}
0   6
1   9
2   0
3   5
4   4
5   0
6   9
7   6
8   2
9   5
10   7
11   2
12   6
13   7
14   6
15   6
16   3
17   4
18   1
0   {'id': '386472', 'name': 'coastguard', 'title': '', 'cover': ''}
1   {'id': '25560', 'name': 'navy', 'title': '', 'cover': ''}
2   {'id': '36677', 'name': 'marines', 'title': '', 'cover': ''}
3   {'id': '108769', 'name': 'airforce', 'title': '', 'cover': ''}
4   {'id': '24740', 'name': 'army', 'title': '', 'cover': ''}
5   {'id': '7221', 'name': 'military', 'title': '', 'cover': ''}
6   {'id': '738782', 'name': 'womeninuniform', 'title': '', 'cover': ''}
7   {'id': '1634905735367685', 'name': 'womeninthemilitary', 'title': '

7   1
8   1
9   6
10   2
11   4
12   6
13   6
14   3
15   7
16   8
17   2
18   5
0   {'id': '11797', 'name': 'f', 'title': '', 'cover': ''}
1   {'id': '5433', 'name': 'friends', 'title': '#FriendshipDay Contest: Create a musical with some #friends and share it on Instagram and here. The best ones get featured.', 'cover': ''}
0   6
1   9
2   0
3   5
4   2
5   9
6   8
7   5
8   1
9   1
10   4
11   7
12   0
13   8
14   9
15   0
16   2
17   4
18   1
0   6
1   9
2   0
3   2
4   8
5   2
6   3
7   5
8   0
9   4
10   3
11   8
12   3
13   6
14   5
15   9
16   2
17   6
18   6
0   6
1   8
2   8
3   7
4   4
5   4
6   3
7   3
8   9
9   8
10   4
11   4
12   6
13   5
14   5
15   0
16   2
17   7
18   3
0   6
1   8
2   9
3   9
4   1
5   0
6   4
7   6
8   4
9   6
10   5
11   4
12   3
13   9
14   6
15   9
16   5
17   3
18   7
0   {'id': '9166', 'name': 'running', 'title': '', 'cover': ''}
1   {'id': '20884', 'name': 'viral', 'title': '', 'cover': ''}
2   {'id': '217185', 'name': 'fitgirl', 'title': '', '

In [9]:
new_youtube_df.to_sql('trending', if_exists='replace', con=engine, index=False) 
engine.execute('select * from trending').fetchall()



[('2kyS6SvSYSE', 'WE WANT TO TALK ABOUT OUR MARRIAGE', 'SHANtell martin', 748374),
 ('1ZAPwfrtAFY', 'The Trump Presidency: Last Week Tonight with John Oliver (HBO)', 'last week tonight trump presidency|"last week tonight donald trump"|"john oliver trump"|"donald trump"', 2418783),
 ('5qpjK5DgCt4', 'Racist Superman | Rudy Mancuso, King Bach & Lele Pons', 'racist superman|"rudy"|"mancuso"|"king"|"bach"|"racist"|"superman"|"love"|"rudy mancuso poo bear black white official music video"|"iphone x by pinea ... (17 characters truncated) ... "hannahstocking"|"rudymancuso"|"inanna"|"anwar"|"sarkis"|"shots"|"shotsstudios"|"alesso"|"anitta"|"brazil"|"Getting My Driver\'s License | Lele Pons"', 3191434),
 ('puqaWrEC7tY', 'Nickelback Lyrics: Real or Fake?', 'rhett and link|"gmm"|"good mythical morning"|"rhett and link good mythical morning"|"good mythical morning rhett and link"|"mythical morning"|"Season ... (179 characters truncated) ... ickelback (musical group)"|"rock"|"music"|"lyrics"|"chad k

In [13]:
new_tiktok_data_df.to_sql('trending', if_exists='append', con=engine, index=False) 
engine.execute('select * from trending').fetchall()

ProgrammingError: (psycopg2.ProgrammingError) can't adapt type 'dict'
[SQL: INSERT INTO trending (platform_id, title, view_count, tags) VALUES (%(platform_id)s, %(title)s, %(view_count)s, %(tags)s)]
[parameters: ({'platform_id': '6907228749016714497', 'title': 'Confidence went 📈', 'view_count': 44800, 'tags': []}, {'platform_id': '6875468410612993286', 'title': 'Quiet Zone... follow me on insta: joeysofo. Comment where you wanna see me blade next. Reply to @dwight_schnuute', 'view_count': 838100, 'tags': []}, {'platform_id': '6898699405898059010', 'title': 'Iphone bend test🤗 #tiktok #viral #fyp #iphone #test #bend', 'view_count': 15300000, 'tags': [{'id': '23428', 'name': 'tiktok', 'title': '', 'cover': ''}, {'id': '20884', 'name': 'viral', 'title': '', 'cover': ''}, {'id': '229207', 'name': 'fy ... (59 characters truncated) ... one', 'title': '', 'cover': ''}, {'id': '15134', 'name': 'test', 'title': '', 'cover': ''}, {'id': '16962', 'name': 'bend', 'title': '', 'cover': ''}]}, {'plat